In [2]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [3]:
mnist= tf.keras.datasets.mnist.load_data()
(train_data,train_label) , (test_data,test_label) = mnist

train_data.shape ,train_label.shape , test_data.shape,test_label.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
encoder = OneHotEncoder(sparse_output=False)

train_label_t = encoder.fit_transform(train_label.reshape(-1,1))
test_label_t = encoder.transform(test_label.reshape(-1,1))

train_label_t.shape ,test_label_t.shape

((60000, 10), (10000, 10))

In [5]:
train_data_t = train_data.reshape(60000,784,)
test_data_t = test_data.reshape(10000,784,)
train_data_t.shape,test_data_t.shape

((60000, 784), (10000, 784))

In [6]:
input_width = 28
input_height = 28 
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
conv1_k =5
conv2_k =5
stride_conv1 = 1
stride_conv2 = 1

max_pool1_k=2 ## i.e 2*2 max pooling on our images  
max_pool2_k=2

n_hidden = 1024
n_out = 10 

input_size_to_hidden = ( input_width//(max_pool1_k*max_pool2_k) )*( input_height//(max_pool1_k*max_pool2_k) )*n_conv2
print(input_size_to_hidden)

3136


### Initialising Weights

In [7]:
random_state = 42
weights ={
    'wc1':tf.Variable(tf.compat.v1.random_normal([conv1_k,conv1_k,input_channels,n_conv1],seed = random_state)),
    'wc2':tf.Variable(tf.compat.v1.random_normal([conv2_k,conv2_k,n_conv1,n_conv2],seed = random_state)),
    'wh1':tf.Variable(tf.compat.v1.random_normal([input_size_to_hidden,n_hidden],seed = random_state)),
    'wo':tf.Variable(tf.compat.v1.random_normal([n_hidden,n_out],seed = random_state)),
}
biases ={
    'bc1':tf.Variable(tf.compat.v1.random_normal([n_conv1],seed = random_state)),
    'bc2':tf.Variable(tf.compat.v1.random_normal([n_conv2],seed = random_state)),
    'bh1':tf.Variable(tf.compat.v1.random_normal([n_hidden],seed = random_state)),
    'bo':tf.Variable(tf.compat.v1.random_normal([n_out],seed = random_state)),
}

In [8]:
def conv(X,weights,bias,strides=1):
    out = tf.nn.conv2d(X,weights,padding='SAME',strides=[1,strides,strides,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

In [9]:
def maxpooling(X,k=2):
    out = tf.nn.max_pool(X,padding='SAME',ksize = [1,k,k,1],strides=[1,k,k,1])
    return out 

In [10]:
## Forward Propagation............

def cnn(X,weights,biases,keep_prob=0.8):
    ## With reshape flattened images.

    X = tf.reshape(X,shape=[-1,input_height,input_width,input_channels])

    conv1 = conv(X,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool , shape=[-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout,keep_prob)
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    
    print('shapes of following are :')
    print('conv1 :',conv1.shape)
    print('conv1_pool :',conv1_pool.shape)
    print('conv2 :',conv2.shape)
    print('conv2_pool :',conv2_pool.shape)
    print('hidden_input :',hidden_input.shape)
    print('hidden_output :',hidden_output.shape)
    print('output :',output.shape)
    return output

In [11]:
X = tf.compat.v1.placeholder('float',[None,input_pixels])
Y = tf.compat.v1.placeholder(tf.int32,[None,n_out])
keep_prob = tf.compat.v1.placeholder('float')

pred  = cnn(X,weights,biases,keep_prob)

shapes of following are :
conv1 : (None, 28, 28, 32)
conv1_pool : (None, 14, 14, 32)
conv2 : (None, 14, 14, 64)
conv2_pool : (None, 7, 7, 64)
hidden_input : (None, 3136)
hidden_output : (None, 1024)
output : (None, 10)


In [12]:
 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred , labels=Y)) ## use v2 function as we will getts some deprecated warnning  

In [13]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.005)
optimize = optimizer.minimize(cost)

In [14]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer()) 

In [ ]:
%%time 
batch_size = 100
n_size = len(train_data_t)
X_train = train_data_t
Y_train = train_label_t
index = np.arange(n_size)

for i in range(1):
    # np.random.shuffle(index)

    # X_train = X_train[index]
    # Y_train = Y_train[index]
    
    total_cost = 0
    for j in range(batch_size , n_size , batch_size):
        
        batch_X = X_train[j-batch_size : j]
        batch_Y = Y_train[j-batch_size : j]
    
        c,_ = sess.run([cost,optimize] , feed_dict={X:batch_X,Y:batch_Y,keep_prob:0.8})
        total_cost += c ## Total cost is better to evealute overall perfromace ..
 
    print(i,':',total_cost)

In [ ]:
Y_pred = tf.argmax(pred,1) # axis=1 
Y_true = tf.argmax(Y,1)

Corrects = tf.equal(Y_pred , Y_true)

Y_train_pred ,correct_train = sess.run( [Y_pred,Corrects] ,feed_dict = {X:train_data_t ,Y:train_label_t,keep_prob:1})
Y_test_pred ,correct_test = sess.run( [Y_pred,Corrects] ,feed_dict = {X:test_data_t ,Y:test_label_t,keep_prob:1})

print("Train Accuracy :",correct_train.mean()) 
print("Test Accuracy :",correct_test.mean()) 